# MP3 태그

지난시간까지 어떤 구조로 MP3 파일안에 가수, 앨범, 가사, 이미지 등이 추가 되는지 배웠으며 이것을 ID3 태그라고 부르는것도 알았습니다. 그리고 그 과정이 그리 녹녹하진 않았을꺼라 생각됩니다. 이제 대강 어떻게 동작하는지는 알았으니 우리가 이것을 직접 구현하지 않고 기존에 훌륭하게 구현된 외부 라이브러리를 사용하여 내 컴퓨터의 MP3 파일에 메타 정보를 자동으로 기록하는 프로그램을 작성해볼까 합니다.

### mutagen [[공식링크]](https://mutagen.readthedocs.io/en/latest/)

Mutagen 은 오디오 메타데이터를 처리하기 위한 파이썬 라이브러리이며 MP3 외에도 FLAC, MP4, Ogg Opus, Ogg FLAC 등 거의 대부분의 오디오 파일을 지원합니다. 기본 ID3v2.4 의 모든 프레임이 분석가능 하며 그 외에도 여러가지 기능이 있습니다. Mutagen 은 리눅스, 윈도우, macOS 에서 구동가능하며 파이썬 3.6 이상에서 동작 합니다.

> pip install mutagen

In [ ]:
import mutagen
m = mutagen.File("bensound-newdawn.mp3")
for k in m.keys():
    print(m.tags.getall(k))

mutagen 라이브러리를 사용하면 위의 코드처럼 아주 간단하게 해당 MP3 파일안에 어떤 ID3 태그가 존재하는지를 확인할 수 있습니다.

### ID3 태그 추가하기

mutagen 에는 ID3v2.3 그리고 IDv2.4 의 모든 프레임을 여러가지 방식으로 아주 간편하게 추가할 수 있습니다.

In [ ]:
from mutagen.id3 import ID3, TIT2, TPE1, TPE2, USLT

lyrics = """
노래가사
테스트
"""
mp3 = ID3("id3_test.mp3")
mp3.add(TIT2(encoding=3, text=u"제목 추가"))
mp3.add(TPE1(encoding=3, text=u"TPE1 추가"))
mp3.add(TPE2(encoding=3, text=u"TPE2 추가"))
mp3.add(USLT(encoding=3, lang=u"kor", desc=u"", text=lyrics))

mp3.save(v2_version=3)

위의 코드에서 처럼 mutagen 에는 기존의 프레임 아이디에 해당하는 객체들을 모두 만들어놓고 그것을 직접 선언하여 사용할 수 있습니다. 물론 해당하는 프레임 아이디와 해당 프레임에 어떤 데이터가 들어가는지 알고 있어야 합니다.

### Easy ID3
Easy ID3 는 기존의 프레임 ID 를 기억하는게 어렵기 때문에 Mutagen 에서 제공하는 더 간편한 ID3 인터페이스 입니다.

In [ ]:
from mutagen.easyid3 import EasyID3
for k in EasyID3.valid_keys.keys():
    print(k)

mp3 = EasyID3("id3_test.mp3")
mp3["title"] = u"제목입니다."
mp3["album"] = u"앨범명입니다."
mp3["composer"] = u"작곡가입니다."
mp3["artist"] = u"아티스트"
mp3["lyricist"] = u"작사가"
mp3["albumartist"] = u"앨범아티스트"
mp3["tracknumber"] = u"10/20"
mp3["genre"] = u"OST"
mp3.save()

mutagen 에서는 기존의 프레임명을 기억하고 사용하는 불편함을 좀 더 편하고 쉽게 사용할 수 있게 하기 위해 EasyID3 라는 인터페이스를 제공합니다. 기존의 프레임 아이디를 직접 사용하는것보다 좀 더 인간친화적인 표현을 제공하나 한글을 사용하는 우리 입장에선 사실 크게 편리함이 느껴지진 않습니다. 참고로 EasyID3 인터페이스는 ID3 의 모든 태그를 제공하는게 아니기 때문에 결국 추가적인 코딩이 필요하니 참고하셔야 할 부분입니다.

### MP3 파일에서 모든 ID3 태그 제거하고 새파일에 기록하기

In [5]:
from io import BytesIO
from mutagen.id3 import ID3, TIT2, TPE1, TPE2

filename = "bensound-newdawn.mp3"
new_filename = "clean_tag.mp3"

org_mp3 = open(filename, "rb")
file_buff = BytesIO()
file_buff.write(org_mp3.read())
org_mp3.close()
file_buff.seek(0)

ID3().delete(file_buff)

mp3 = ID3()
mp3.add(TIT2(encoding=3, text=u"제목 추가"))
mp3.add(TPE1(encoding=3, text=u"TPE1 추가"))
mp3.add(TPE2(encoding=3, text=u"TPE2 추가"))

with open(new_filename, "wb") as f:
    f.write(file_buff.getbuffer())

mp3.save(new_filename, v2_version=3)

기본적으로 mutagen 의 ID3 태그의 수정 추가는 현재의 파일객체를 대상으로 하기 때문에 만약 MP3 파일에서 태그를 제거하고 새로운 태그를 생성한 후 새로운 파일에 작성하기 위해선 먼저 기존의 MP3 파일을 읽어 들인 후에 메모리상에서 작업을 수행하고 file 객체를 통해 저장해야 합니다.

ID3객체의 ```delete()``` 함수의 <a href="https://mutagen.readthedocs.io/en/latest/api/id3.html?highlight=delete#mutagen.id3.ID3.delete">공식 문서</a>를 살펴보면 

> delete(filething=None, delete_v1=True, delete_v2=True)

첫번째 매개변수인 filething 은 파일이름 혹은 파일 객체를 넘겨줄 수 있게 되어있습니다. ID3 태그뿐 아니라 mutagen 라이브러리의 대부분의 객체들은 매개변수 filething 으로 되어있어 파일 스트림 객체를 넘겨줄수 있다는걸 알 수 있습니다.
<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
org_mp3 = open(filename, "rb")
file_buff = BytesIO()
file_buff.write(org_mp3.read())
org_mp3.close()
file_buff.seek(0)
</pre>

먼저 위의 코드처럼 일단 기존 MP3 파일을 open 한 후 BytesIO() 객체를 통해 메모리에 담습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
ID3().delete(file_buff)
</pre>
mutagen 의 ID3 객체의 ```delete()``` 함수를 통해 위에서 MP3 파일을 메모리로 옮겨놓은 BytesIO 형태의 ```file_buff``` 변수에 담긴 파일데이터에서 ID3 태그를 삭제합니다. 


<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
mp3 = ID3()
mp3.add(TIT2(encoding=3, text=u"제목 추가"))
mp3.add(TPE1(encoding=3, text=u"TPE1 추가"))
mp3.add(TPE2(encoding=3, text=u"TPE2 추가"))
</pre>

일반적으로는 ```ID3(filething)``` 으로 기존의 MP3를 여는게 보통이지만 여기서는 ```mp3 = ID3()``` 으로 새로운 빈 ID3 태그 객체를 생성한 후 ```mp3.add(TIT2(encoding=3, text=u"제목 추가"))``` 처럼 ID3 태그를 추가합니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
with open(new_filename, "wb") as f:
    f.write(file_buff.getbuffer())
</pre>
여기서 ```new_filename``` 이름으로 파일을 저장하는데 아까 위에서 ID3 태그를 모두 삭제한 ```file_buff``` 의 내용으로 파일을 저장합니다. 이렇게 저장된 ```new_filename``` 은 ID3 태그가 모두 삭제된 파일입니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
mp3.save(new_filename, v2_version=3)
</pre>
```ID3()``` 객체를 저장할때는 파일 객체에 저장해야 하는데 우리는 빈 ```ID3()``` 태그를 생성했기 때문에 그냥 저장할수가 없습니닫. 그래서 방금 ID3 태그가 삭제된 파일을 먼저 저장하고 최종적으로 ```ID3()``` 로 생성된 새로운 태그를 ```new_filename``` 에 덮어씌웁니다. 물론 이 방법은 기존의 파일에서 ID3 태그를 제거하고 "새로운 이름" 의 파일로 생성하기 위함이지 기존의 MP3 의 ID3 를 수정하거나 할때는 이렇게 번거로운 방법을 사용할 필요가 없습니다.